<a href="https://colab.research.google.com/github/sundarjhu/UACJ_Jornada2021/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import warnings, numpy as np
warnings.filterwarnings("ignore", category = np.VisibleDeprecationWarning)
try:
  from astroquery.sdss import SDSS
except:
  !pip install astroquery
  from astroquery.sdss import SDSS
from astropy import coordinates as coords
#pos = coords.SkyCoord('202.45748d 29.579716d', frame = 'icrs')
pos = coords.SkyCoord.from_name("KISSR 298")
xid = SDSS.query_region(pos, spectro = True)
sp = SDSS.get_spectra(matches = xid)

In [51]:
sp[0][1].data

FITS_rec([(13.479021, 3.5816, 0.06797916, 0, 0, 1.1088611 , 9.002381, 13.217207),
          (19.215475, 3.5817, 0.06977836, 0, 0, 1.108475  , 9.212716, 13.737914),
          (14.267138, 3.5818, 0.07592517, 0, 0, 1.108091  , 9.300478, 13.168895),
          ...,
          ( 8.162261, 3.9645, 0.35342726, 0, 0, 0.7729795 , 4.512267, 12.689797),
          (11.752623, 3.9646, 0.        , 0, 0, 0.77300555, 4.811207, 12.891481),
          (15.343176, 3.9647, 0.28280747, 0, 0, 0.77303153, 4.932665, 12.677738)],
         dtype=(numpy.record, [('flux', '>f4'), ('loglam', '>f4'), ('ivar', '>f4'), ('and_mask', '>i4'), ('or_mask', '>i4'), ('wdisp', '>f4'), ('sky', '>f4'), ('model', '>f4')]))

In [52]:
len(sp[0])

(56, '/tmp/tmpy0klnhm7')

#**Download the HST images from the Mikulski Archive at Space Telescope (MAST)**
###We will create a temporary directory to store and access these images. This folder will be deleted at the end of the session with the final command.

In [54]:
from astroquery.mast import Observations
import os, requests, shutil, tempfile
#Create temporary folder if it doesn't already exist.
if not(os.path.exists("/content/UACJ/")):
  %mkdir "/content/UACJ/"
#Get relevant information for the images
obs_table = Observations.query_criteria(project = "HST", target_name = "KISSR298", proposal_id = 12951, instrument_name = "WFC3/UVIS")
print(obs_table['filters', 'dataURL'])
"""
  Download each of the files in the resulting table.
  For speed, the file is first read as a stream using the `requests` module into the temporary storage.
  It is then moved to the folder created as part of this session.
"""
url = "https://mast.stsci.edu/api/v0.1/Download/file?uri=" #Base url to MAST
for dataURL, filter in zip(obs_table['dataURL'], obs_table['filters']):
  temp_filename = tempfile.gettempdir() + dataURL.split('/')[-1]
  with requests.get(url + dataURL, stream = True) as r:
    with open(temp_filename, 'wb') as f:
      shutil.copyfileobj(r.raw, f)
  print("Downloading data for filter {} into {}".format(filter, '/content/UACJ/KISSR298_' + filter + '_drz.fits'))
  shutil.move(temp_filename, '/content/UACJ/KISSR298_' + filter + '_drz.fits', copy_function = shutil.copy)

filters               dataURL              
------- -----------------------------------
  F621M mast:HST/product/ic30a2010_drz.fits
 FQ508N mast:HST/product/ic30a2020_drz.fits
  F680N mast:HST/product/ic30a2030_drz.fits


In [55]:
%ls UACJ/

KISSR298_F621M_drz.fits  KISSR298_F680N_drz.fits  KISSR298_FQ508N_drz.fits


#**Construct RGB composite from images**
##Code to construct the composite

In [56]:
from astropy.table import Table
t = Table.read('/content/UACJ/KISSR298_F621M_drz.fits', format = 'fits')

#**Wrapping up**
##When all the analysis is done, the following command will delete the temporary folder.

In [57]:
%rm -r /content/UACJ/